In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

In [ ]:
import matplotlib.pyplot as plt
img = X_train[0]
img_reshaped = img.reshape(28,28)
img_reshaped.shape

plt.figure(figsize=(4,4))
plt.title('sample of ' + str(y_train[0]))
plt.imshow(img_reshaped, cmap='gray')
plt.show()

In [ ]:
class Network(object):
    def __init__(self, n_layers):
        self.params = []

        self.W1 = tf.Variable(
            tf.random.normal([n_layers[0],n_layers[1]],stddev=0.1), name='W1'
        )
        self.b1 = tf.Variable(tf.zeros([1,n_layers[1]]))

        self.W2 = tf.Variable(
			tf.random.normal([n_layers[1], n_layers[2]], stddev=0.1), name='W2'
		)
        self.b2 = tf.Variable(tf.zeros([1,n_layers[2]]))

        self.W3 = tf.Variable(
			tf.random.normal([n_layers[2], n_layers[3]],stddev=0.1), name='W3'
        )
        self.b3 = tf.Variable(tf.zeros([1, n_layers[3]]))

        self.params = [self.W1, self.b1, self.W2, self.b2, self.W3, self.b3]

    def forward(self, x):
        X_tf = tf.cast(x, dtype=tf.float32)
        Z1 = tf.matmul(X_tf, self.W1) + self.b1
        Z1 = tf.nn.relu(Z1)
        Z2 = tf.matmul(Z1, self.W2) + self.b2
        Z2 = tf.nn.relu(Z2)
        Z3 = tf.matmul(Z2, self.W3) + self.b3
        # Y = tf.nn.sigmoid(Z3)
        return Z3

    def loss(self, y_true, logits):
        y_true_tf = tf.cast(tf.reshape(y_true, (-1,1)), dtype = tf.float32)
        logits_tf = tf.cast(tf.reshape(logits, (-1,1)), dtype = tf.float32)
        return tf.nn.sigmoid_cross_entropy_with_logits(y_true_tf,logits_tf) 

    def backward(self, x, y):
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
        with tf.GradientTape() as tape:
            predicted = self.forward(x)
            current_loss = self.loss(y, predicted)
        grads = tape.gradient(current_loss, self.params)
        optimizer.apply_gradients(zip(grads, self.params))


In [ ]:
n_layers = [784, 100, 30, 10]
epochs = 100
X_reshape = X_train.reshape(60000,784).astype('float32')/255.0
y_one_hot = tf.one_hot(y_train, depth = 10)

net = Network(n_layers)

In [ ]:
from sklearn.metrics import precision_score, f1_score
for epoch in range(epochs):
    net.backward(X_reshape,y_one_hot)
    acc =f1_score(y_true = tf.math.argmax(y_one_hot,1), y_pred = tf.math.argmax(net.forward(X_reshape),1),average='micro')
    if epoch % 10 == 0:
        print('accuracy score : {score}'.format(score = acc * 100))
